In [ ]:
# Research Project Site 1 VDM Training

 # 1) loading the annotation project data and viewing the data

In [6]:
!pip install soundevent

   ---------------------------------------- 0.0/131.2 kB ? eta -:--:--
   ---------------------------------------- 0.0/131.2 kB ? eta -:--:--
   ---------------------------------------- 0.0/131.2 kB ? eta -:--:--
   ---------------------------------------- 0.0/131.2 kB ? eta -:--:--
   ---------------------------------------- 0.0/131.2 kB ? eta -:--:--
   ---------------------------------------- 0.0/131.2 kB ? eta -:--:--
   ---------------------------------------- 0.0/131.2 kB ? eta -:--:--
   ---------------------------------------- 0.0/131.2 kB ? eta -:--:--
   ------------ -------------------------- 41.0/131.2 kB 991.0 kB/s eta 0:00:01
   ------------ -------------------------- 41.0/131.2 kB 991.0 kB/s eta 0:00:01
   ------------------ -------------------- 61.4/131.2 kB 469.7 kB/s eta 0:00:01
   ------------------ -------------------- 61.4/131.2 kB 469.7 kB/s eta 0:00:01
   --------------------- ----------------- 71.7/131.2 kB 302.7 kB/s eta 0:00:01
   --------------------- -------

In [2]:
import json
from pathlib import Path

In [52]:
from soundevent import data, io

In [8]:
from collections import defaultdict

In [11]:
!pip install audioclass

In [22]:
!pip install tflite-runtime!pip install tensorflow

ERROR: Could not find a version that satisfies the requirement tflite-runtime (from versions: none)
ERROR: No matching distribution found for tflite-runtime


In [10]:
from tensorflow._api.v2.lite import Interpreter

In [11]:
from audioclass import SimpleIterator
from audioclass.models.birdnet import BirdNET
import pandas as pd
from tqdm import tqdm

In [14]:
vdm_annotation_path = Path("SM1_6s_2024-07-10T11_22_51.505766.json")
with open(vdm_annotation_path) as file:
    annotation_contents = json.load(file)

print(json.dumps(annotation_contents, indent=2))

{
  "version": "1.1.0",
  "created_on": "2024-07-10T11:22:51.505766",
  "data": {
    "uuid": "97817b89-4560-422e-9338-60f1a005cbfb",
    "collection_type": "annotation_project",
    "users": [
      {
        "uuid": "63e6952b-1e95-440b-9fbb-6e0aa03b4ecb",
        "username": "hhhyt",
        "email": "yuting.huang.23@ucl.ac.uk",
        "name": "Yuting Huang",
        "institution": null
      }
    ],
    "tags": [
      {
        "id": 0,
        "key": "sounds",
        "value": "parrot"
      },
      {
        "id": 1,
        "key": "sound",
        "value": "human"
      },
      {
        "id": 2,
        "key": "sound",
        "value": "bird"
      },
      {
        "id": 3,
        "key": "n/a",
        "value": "nothing"
      },
      {
        "id": 4,
        "key": "sound",
        "value": "unknown"
      },
      {
        "id": 5,
        "key": "environment",
        "value": "rain or wind"
      },
      {
        "id": 6,
        "key": "environment",
        "

In [16]:
vdm_sample = io.load(vdm_annotation_path, type="annotation_project")
print(repr(vdm_sample))

AnnotationProject(created_on=datetime.datetime(2024, 5, 23, 15, 53, 46, 549162), name='SM1_6s', description='the 6s clips for sm01', instructions='2 clips of 6s extracted from each recording')


In [18]:
for clip_annotation in vdm_sample.clip_annotations:
    clip = clip_annotation.clip
    recording = clip.recording
    print(
        f"* Recording {recording.path} [from "
        f"{clip.start_time:.3f}s to {clip.end_time:.3f}s]"
    )
    print(
        f"\t{len(clip_annotation.sound_events)} sound event annotations found"
    )
    for annotation in clip_annotation.sound_events:
        sound_event = annotation.sound_event
        print(f"\t+ Sound event ")
        for tag in annotation.tags:
            print(f"\t\t- {tag}")
    print("")

* Recording C:\Users\dorah\OneDrive\妗岄潰\seychelles_parrot_data\SM01 (VDM1)\2020\SM01_02_2020\SM01_20200227_130000.wav [from 66.000s to 72.000s]
	0 sound event annotations found

* Recording C:\Users\dorah\OneDrive\妗岄潰\seychelles_parrot_data\SM01 (VDM1)\2020\SM01_02_2020\SM01_20200213_120000.wav [from 114.000s to 120.000s]
	0 sound event annotations found

* Recording C:\Users\dorah\OneDrive\妗岄潰\seychelles_parrot_data\SM01 (VDM1)\2020\SM01_03_2020\SM01_20200306_110000.wav [from 264.000s to 270.000s]
	0 sound event annotations found

* Recording C:\Users\dorah\OneDrive\妗岄潰\seychelles_parrot_data\SM01 (VDM1)\2020\SM01_02_2020\SM01_20200207_090000.wav [from 204.000s to 210.000s]
	0 sound event annotations found

* Recording C:\Users\dorah\OneDrive\妗岄潰\seychelles_parrot_data\SM01 (VDM1)\2020\SM01_03_2020\SM01_20200302_140000.wav [from 222.000s to 228.000s]
	0 sound event annotations found

* Recording C:\Users\dorah\OneDrive\妗岄潰\seychelles_parrot_data\SM01 (VDM1)\2020\SM01_02_2020\SM01_2020

In [41]:
# save the annotation project
io.save(vdm_sample, "vdm_sample.json")

In [96]:
# 2) select the completely annotated files

In [20]:
# create functions
def load_annotation_file(annotation_file):
    return io.load(annotation_file, type="annotation_project")


def has_issue(task):
    for badge in task.status_badges:
        if badge.state == data.AnnotationState.rejected:
            return True

    return False


def is_complete(task):
    for badge in task.status_badges:
        if badge.state == data.AnnotationState.completed:
            return True

    return False


def select_clips(annotation_project, complete=None, issue=None):
    tasks = [task for task in annotation_project.tasks]

    if complete is not None:
        tasks = [task for task in tasks if is_complete(task) == complete]

    if issue is not None:
        tasks = [task for task in tasks if has_issue(task) == issue]

    clips = {task.clip.uuid for task in tasks}
    return [
        clip_annotation
        for clip_annotation in annotation_project.clip_annotations
        if clip_annotation.clip.uuid in clips
    ]

path = "vdm_sample.json"
annotation_project = load_annotation_file(path)
complete_clips = select_clips(annotation_project, complete=True, issue=False)

In [22]:
# view the completely annotated clips
for clip_annotation in complete_clips:
    clip = clip_annotation.clip
    recording = clip.recording
    print(
        f"* Recording {recording.path} [from "
        f"{clip.start_time:.3f}s to {clip.end_time:.3f}s]"
    )

* Recording C:\Users\dorah\OneDrive\妗岄潰\seychelles_parrot_data\SM01 (VDM1)\2020\SM01_02_2020\SM01_20200227_130000.wav [from 66.000s to 72.000s]
* Recording C:\Users\dorah\OneDrive\妗岄潰\seychelles_parrot_data\SM01 (VDM1)\2020\SM01_02_2020\SM01_20200213_120000.wav [from 114.000s to 120.000s]
* Recording C:\Users\dorah\OneDrive\妗岄潰\seychelles_parrot_data\SM01 (VDM1)\2020\SM01_03_2020\SM01_20200306_110000.wav [from 264.000s to 270.000s]
* Recording C:\Users\dorah\OneDrive\妗岄潰\seychelles_parrot_data\SM01 (VDM1)\2020\SM01_02_2020\SM01_20200207_090000.wav [from 204.000s to 210.000s]
* Recording C:\Users\dorah\OneDrive\妗岄潰\seychelles_parrot_data\SM01 (VDM1)\2020\SM01_03_2020\SM01_20200302_140000.wav [from 222.000s to 228.000s]
* Recording C:\Users\dorah\OneDrive\妗岄潰\seychelles_parrot_data\SM01 (VDM1)\2020\SM01_02_2020\SM01_20200219_090000.wav [from 42.000s to 48.000s]
* Recording C:\Users\dorah\OneDrive\妗岄潰\seychelles_parrot_data\SM01 (VDM1)\2020\SM01_03_2020\SM01_20200311_170000.wav [from 264.

In [24]:
len(complete_clips) 

990

In [26]:
# 3) Extract features
 # Load latest annotations
annotation_project = io.load("vdm_sample.json")

In [28]:
# Load the birdnet model into the notebook!
model = BirdNET.load()

In [30]:
# as all 990 clips have been completely annotated, no need to remove any clips from the original json
selected_clip_annotations = annotation_project.clip_annotations[:990]

In [32]:
prefix = Path.home() / "OneDrive" / "桌面" / "seychelles_parrot_data"
print(f"Prefix exists: {prefix.exists()}")
audio_dir = Path("SM01 (VDM1)")
print(f"audio dir: {audio_dir.exists()}")

Prefix exists: True
audio dir: False


In [34]:
# print out some of the file paths to check
for clip_annotation in selected_clip_annotations[:10]: # just the first 10
    clip = clip_annotation.clip
    recording = clip.recording

    print(recording.path)

C:\Users\dorah\OneDrive\妗岄潰\seychelles_parrot_data\SM01 (VDM1)\2020\SM01_02_2020\SM01_20200227_130000.wav
C:\Users\dorah\OneDrive\妗岄潰\seychelles_parrot_data\SM01 (VDM1)\2020\SM01_02_2020\SM01_20200213_120000.wav
C:\Users\dorah\OneDrive\妗岄潰\seychelles_parrot_data\SM01 (VDM1)\2020\SM01_03_2020\SM01_20200306_110000.wav
C:\Users\dorah\OneDrive\妗岄潰\seychelles_parrot_data\SM01 (VDM1)\2020\SM01_02_2020\SM01_20200207_090000.wav
C:\Users\dorah\OneDrive\妗岄潰\seychelles_parrot_data\SM01 (VDM1)\2020\SM01_03_2020\SM01_20200302_140000.wav
C:\Users\dorah\OneDrive\妗岄潰\seychelles_parrot_data\SM01 (VDM1)\2020\SM01_02_2020\SM01_20200219_090000.wav
C:\Users\dorah\OneDrive\妗岄潰\seychelles_parrot_data\SM01 (VDM1)\2020\SM01_03_2020\SM01_20200311_170000.wav
C:\Users\dorah\OneDrive\妗岄潰\seychelles_parrot_data\SM01 (VDM1)\2020\SM01_03_2020\SM01_20200306_110000.wav
C:\Users\dorah\OneDrive\妗岄潰\seychelles_parrot_data\SM01 (VDM1)\2020\SM01_02_2020\SM01_20200216_130000.wav
C:\Users\dorah\OneDrive\妗岄潰\seychelles_parrot_

In [39]:
print(recording.path.exists())

False


In [36]:
# as there were libs and file error when trying to view the example clip
# We need to fix the paths in the recordings
prefix = Path('/Users/dorah/OneDrive/妗岄潰/seychelles_parrot_data/')
audio_dir = Path("/SM01 (VDM1)")

from pathlib import Path

for clip_annotation in selected_clip_annotations:
    clip = clip_annotation.clip
    recording = clip.recording

    # Only fix the path if not previously done
    recording.path = Path(str(recording.path).replace("妗岄潰","桌面"))

    if not recording.path.exists():
        raise ValueError(f"File not found {recording.path}")

In [48]:
# view the infos using an example clip
example_clip = selected_clip_annotations[0].clip

print("Clips contain info about when they start and end: ", clip.start_time, clip.end_time)

example_outputs = model.process_clip(example_clip)

print("The output contains two processed clips", len(example_outputs))

example_processed_clip = example_outputs[0]

print("Processed clips might be a subclip of the input: ", example_processed_clip.clip.start_time, example_processed_clip.clip.end_time) 

print("Processed clips can contain extracted features. In this case: ", len(example_processed_clip.features))

print("The first feature:  ", example_processed_clip.features[0])

Clips contain info about when they start and end:  282.0 288.0
The output contains two processed clips 2
Processed clips might be a subclip of the input:  66.0 69.0
Processed clips can contain extracted features. In this case:  1024
The first feature:   name='BirdNET_0' value=0.22874562442302704


In [38]:
def extract_tags_info(tags):
    return {tag.value: True for tag in tags}


def extract_task_info(annotation_task):
    ready = False
    issues = False

    for badge in annotation_task.status_badges:
        if badge.state == data.AnnotationState.completed:
            ready = True
            continue

        if badge.state == data.AnnotationState.rejected:
            issues = True
            continue

    return {"ready": ready, "issues": issues}


def extract_clip_annotation_info(clip_annotation):
    return {
        "annotation_id": str(clip_annotation.uuid),
        **extract_tags_info(clip_annotation.tags),
    }


def extract_feature_info(processed_clip):
    return {
        "path": processed_clip.clip.recording.path,
        "clip_id": processed_clip.clip.uuid,
        "start_time": processed_clip.clip.start_time,
        "end_time": processed_clip.clip.end_time,
        **{feature.name: feature.value for feature in processed_clip.features},
    }


# Process all clips in a loop
outputs = []
for clip_annotation in tqdm(selected_clip_annotations):
    clip = clip_annotation.clip

    # Its a good idea to wrap this processing bit in a try-except block
    # so that we can safely skip the clips that failed to process.
    try:
        processed_clips = model.process_clip(clip)

        # NOTE: If the clip is long the model will split it into multiple clips
        # and return a list of processed clips.
        for processed_clip in processed_clips:
            outputs.append(
                {
                    **extract_feature_info(processed_clip),
                    **extract_clip_annotation_info(clip_annotation),
                }
            )

    except Exception as e:
        print(f"Failed to process clip: {clip}. Error: {e}")
        continue

100%|████████████████████████████████████████████████████████████████████████████████| 990/990 [02:30<00:00,  6.57it/s]


In [64]:
# save the outputs into a data frame
df = pd.DataFrame(outputs)

In [66]:
df

,path,clip_id,start_time,end_time,BirdNET_0,BirdNET_1,BirdNET_2,BirdNET_3,BirdNET_4,BirdNET_5,...,parrot,human,bird,nothing,unknown,rain or wind,drops,bulbul,chat,sunbird
0,C:\Users\dorah\OneDrive\桌面\seychelles_parrot_d...,3242e170-32f3-4fa9-9a71-c7381c756520,66.0,69.0,0.228746,0.019846,0.012975,0.026412,0.211450,0.197363,...,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,C:\Users\dorah\OneDrive\桌面\seychelles_parrot_d...,ed320053-71b1-41ed-8060-319d4394d8dc,69.0,72.0,0.219624,0.000000,0.216945,0.085611,1.225403,0.833601,...,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,C:\Users\dorah\OneDrive\桌面\seychelles_parrot_d...,c03831f2-fce4-46e3-aeb0-17b497e054de,114.0,117.0,0.178642,0.000000,0.808444,0.000075,0.424041,1.177127,...,NaN,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,C:\Users\dorah\OneDrive\桌面\seychelles_parrot_d...,39648b99-a3ad-4ac6-89cd-96f7f209cf51,117.0,120.0,0.130440,0.000000,1.027665,0.522229,1.207054,0.572254,...,NaN,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,C:\Users\dorah\OneDrive\桌面\seychelles_parrot_d...,33b16505-5923-4d9f-9ac9-48a45d0162c4,264.0,267.0,0.000000,0.475184,0.056921,1.339753,0.585905,0.000000,...,NaN,NaN,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1975,C:\Users\dorah\OneDrive\桌面\seychelles_parrot_d...,86954731-578e-4f86-874a-5fd953ea5370,9.0,12.0,0.183355,0.000000,0.163335,0.007175,0.551006,0.343138,...,NaN,NaN,NaN,True,NaN,True,NaN,NaN,NaN,NaN
1976,C:\Users\dorah\OneDrive\桌面\seychelles_parrot_d...,61ebfa42-7e01-435b-8d82-d965f1275eb0,108.0,111.0,0.087861,0.107960,0.408588,0.201990,0.860113,0.500617,...,NaN,NaN,NaN,True,NaN,NaN,NaN,NaN,NaN,NaN
1977,C:\Users\dorah\OneDrive\桌面\seychelles_parrot_d...,52b4b97f-e1b5-417d-9d93-f8d9784cc138,111.0,114.0,0.496575,0.356684,0.240230,0.261427,0.682009,0.638558,...,NaN,NaN,NaN,True,NaN,NaN,NaN,NaN,NaN,NaN
1978,C:\Users\dorah\OneDrive\桌面\seychelles_parrot_d...,cdd65377-4f0e-491f-876d-977a547b61e5,282.0,285.0,0.059039,0.000000,0.000000,0.000000,0.436419,0.442027,...,NaN,NaN,NaN,True,NaN,NaN,NaN,NaN,NaN,NaN


In [68]:
df.to_csv("full_vdm_features.csv")

In [90]:
print(df.columns)

Index(['path', 'clip_id', 'start_time', 'end_time', 'BirdNET_0', 'BirdNET_1',
       'BirdNET_2', 'BirdNET_3', 'BirdNET_4', 'BirdNET_5',
       ...
       'parrot', 'human', 'bird', 'nothing', 'unknown', 'rain or wind',
       'drops', 'bulbul', 'chat', 'sunbird'],
      dtype='object', length=1039)


In [82]:
# compute the average features for two subclips
full_vdm_features = pd.read_csv('full_vdm_features.csv', low_memory=False)

In [124]:
full_vdm_features.drop(columns = ['clip_id','Unnamed: 0'])

,path,start_time,end_time,BirdNET_0,BirdNET_1,BirdNET_2,BirdNET_3,BirdNET_4,BirdNET_5,BirdNET_6,...,parrot,human,bird,nothing,unknown,rain or wind,drops,bulbul,chat,sunbird
0,C:\Users\dorah\OneDrive\桌面\seychelles_parrot_d...,66.0,69.0,0.228746,0.019846,0.012975,0.026412,0.211450,0.197363,0.000000,...,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,C:\Users\dorah\OneDrive\桌面\seychelles_parrot_d...,69.0,72.0,0.219624,0.000000,0.216945,0.085611,1.225403,0.833601,0.123097,...,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,C:\Users\dorah\OneDrive\桌面\seychelles_parrot_d...,114.0,117.0,0.178642,0.000000,0.808444,0.000075,0.424041,1.177127,0.000000,...,NaN,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,C:\Users\dorah\OneDrive\桌面\seychelles_parrot_d...,117.0,120.0,0.130440,0.000000,1.027665,0.522229,1.207054,0.572254,0.000000,...,NaN,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,C:\Users\dorah\OneDrive\桌面\seychelles_parrot_d...,264.0,267.0,0.000000,0.475184,0.056921,1.339753,0.585905,0.000000,0.000000,...,NaN,NaN,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1975,C:\Users\dorah\OneDrive\桌面\seychelles_parrot_d...,9.0,12.0,0.183355,0.000000,0.163335,0.007175,0.551006,0.343138,0.103384,...,NaN,NaN,NaN,True,NaN,True,NaN,NaN,NaN,NaN
1976,C:\Users\dorah\OneDrive\桌面\seychelles_parrot_d...,108.0,111.0,0.087861,0.107960,0.408588,0.201990,0.860113,0.500617,0.647116,...,NaN,NaN,NaN,True,NaN,NaN,NaN,NaN,NaN,NaN
1977,C:\Users\dorah\OneDrive\桌面\seychelles_parrot_d...,111.0,114.0,0.496575,0.356684,0.240230,0.261427,0.682009,0.638558,1.008194,...,NaN,NaN,NaN,True,NaN,NaN,NaN,NaN,NaN,NaN
1978,C:\Users\dorah\OneDrive\桌面\seychelles_parrot_d...,282.0,285.0,0.059039,0.000000,0.000000,0.000000,0.436419,0.442027,0.000000,...,NaN,NaN,NaN,True,NaN,NaN,NaN,NaN,NaN,NaN


In [126]:

# Define columns that are categorical (factors)
categorical_columns = ['path', 'parrot','human', 'bird', 'nothing', 'unknown', 'rain or wind',
       'drops', 'bulbul', 'chat', 'sunbird']  # Add any other categorical columns here

# Identify numeric columns for averaging
numeric_columns = full_vdm_features.select_dtypes(include=[float, int]).columns

# Remove categorical and time columns from numeric columns list if they are included
numeric_columns = [col for col in numeric_columns if col not in categorical_columns + ['start_time', 'end_time','annotation_id']]

# Group by 'annotation_id' and compute the average of each numeric feature column
fused_features_numeric = full_vdm_features.groupby('annotation_id')[numeric_columns].mean().reset_index()

# For categorical columns, we can keep the first instance (assuming consistency)
# Exclude 'annotation_id' here to avoid duplication
fused_features_categorical = full_vdm_features.groupby('annotation_id')[categorical_columns].first().reset_index()

# For start_time and end_time,custom logic
start_times = full_vdm_features.groupby('annotation_id')['start_time'].first().reset_index()
end_times = full_vdm_features.groupby('annotation_id')['end_time'].last().reset_index()

# Merge numeric, categorical, and time DataFrames
fused_features = fused_features_numeric.merge(fused_features_categorical, on='annotation_id')
fused_features = fused_features.merge(start_times, on='annotation_id')
fused_features = fused_features.merge(end_times, on='annotation_id')


In [130]:
fused_features.drop(columns = ['Unnamed: 0'])

,annotation_id,BirdNET_0,BirdNET_1,BirdNET_2,BirdNET_3,BirdNET_4,BirdNET_5,BirdNET_6,BirdNET_7,BirdNET_8,...,bird,nothing,unknown,rain or wind,drops,bulbul,chat,sunbird,start_time,end_time
0,005bee3c-db54-41a4-bd47-fd9d10c18abf,0.000000,0.208666,0.879510,0.563163,0.618243,0.360183,0.000000,0.007118,0.364522,...,None,None,True,None,None,None,None,None,156.0,162.0
1,00b820f6-570b-4fdb-be0e-260ff41c19f9,0.085495,0.000000,0.257731,0.000000,0.362913,1.260825,0.331117,0.000000,0.000000,...,True,None,True,None,None,None,None,None,198.0,204.0
2,00de27ef-815d-4755-9d74-cf828cca2a74,0.064180,0.025133,0.502210,0.135793,0.899278,0.255933,0.046089,0.000000,0.000000,...,None,None,True,None,None,None,None,None,264.0,270.0
3,00f3b2a4-dbfa-4ed2-bb64-0f472586a6f8,0.343786,0.131952,0.480987,0.000000,0.480376,1.396650,0.150497,0.080120,0.181611,...,True,None,None,None,None,None,None,None,0.0,6.0
4,012a4978-e012-4177-80a1-071ea2b75506,0.112048,0.058986,0.726223,0.341798,0.730211,0.915552,0.000000,0.020937,0.096703,...,True,None,True,None,None,None,None,None,162.0,168.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
985,fe570002-de0f-4a9f-86ff-0a710bf8242d,0.195892,0.061640,0.288349,0.143575,0.259042,0.259570,0.575948,0.000000,0.064728,...,None,True,None,True,None,None,None,None,138.0,144.0
986,ff07fcb7-aa3b-4a2c-af70-196488a053cd,0.082896,0.000000,1.255835,0.210454,0.515464,0.999544,0.025532,0.000000,0.045377,...,True,None,None,None,None,None,None,None,192.0,198.0
987,ff8b2dae-7553-4826-80bd-fb9766542ddd,0.200742,0.000000,0.493353,0.109698,1.152392,0.015637,0.000000,0.000000,0.000000,...,None,None,True,None,None,None,None,None,126.0,132.0
988,ff9b1b06-4f99-43fe-b484-5a8e36e4e38c,0.204104,0.069148,1.047245,0.001327,0.751338,0.735946,0.010203,0.095548,0.060450,...,True,None,None,None,None,None,None,None,174.0,180.0


In [132]:
fused_features.to_csv("full_vdm_features.csv")

In [134]:
fused_features.columns

Index(['annotation_id', 'Unnamed: 0', 'BirdNET_0', 'BirdNET_1', 'BirdNET_2',
       'BirdNET_3', 'BirdNET_4', 'BirdNET_5', 'BirdNET_6', 'BirdNET_7',
       ...
       'bird', 'nothing', 'unknown', 'rain or wind', 'drops', 'bulbul', 'chat',
       'sunbird', 'start_time', 'end_time'],
      dtype='object', length=1039)

In [ ]:
# dropping unwanted columns and add site information for the ml development

In [138]:
ml_features = fused_features.drop(columns = ['human','bird','nothing', 'unknown', 'rain or wind', 'drops', 'bulbul', 'chat',
       'sunbird'])

In [144]:
ml_features['site']='vdm'
ml_features

,annotation_id,Unnamed: 0,BirdNET_0,BirdNET_1,BirdNET_2,BirdNET_3,BirdNET_4,BirdNET_5,BirdNET_6,BirdNET_7,...,BirdNET_1019,BirdNET_1020,BirdNET_1021,BirdNET_1022,BirdNET_1023,path,parrot,start_time,end_time,site
0,005bee3c-db54-41a4-bd47-fd9d10c18abf,534.5,0.000000,0.208666,0.879510,0.563163,0.618243,0.360183,0.000000,0.007118,...,0.439383,0.198481,0.000000,0.000000,1.080668,C:\Users\dorah\OneDrive\桌面\seychelles_parrot_d...,True,156.0,162.0,vdm
1,00b820f6-570b-4fdb-be0e-260ff41c19f9,246.5,0.085495,0.000000,0.257731,0.000000,0.362913,1.260825,0.331117,0.000000,...,0.496131,0.258109,1.086174,0.876585,1.709812,C:\Users\dorah\OneDrive\桌面\seychelles_parrot_d...,None,198.0,204.0,vdm
2,00de27ef-815d-4755-9d74-cf828cca2a74,292.5,0.064180,0.025133,0.502210,0.135793,0.899278,0.255933,0.046089,0.000000,...,0.114036,0.410840,0.000000,0.034450,1.708347,C:\Users\dorah\OneDrive\桌面\seychelles_parrot_d...,None,264.0,270.0,vdm
3,00f3b2a4-dbfa-4ed2-bb64-0f472586a6f8,380.5,0.343786,0.131952,0.480987,0.000000,0.480376,1.396650,0.150497,0.080120,...,0.057120,0.080036,0.212184,0.460142,1.017071,C:\Users\dorah\OneDrive\桌面\seychelles_parrot_d...,None,0.0,6.0,vdm
4,012a4978-e012-4177-80a1-071ea2b75506,562.5,0.112048,0.058986,0.726223,0.341798,0.730211,0.915552,0.000000,0.020937,...,0.399708,0.350088,0.521975,0.029837,2.228952,C:\Users\dorah\OneDrive\桌面\seychelles_parrot_d...,None,162.0,168.0,vdm
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
985,fe570002-de0f-4a9f-86ff-0a710bf8242d,1930.5,0.195892,0.061640,0.288349,0.143575,0.259042,0.259570,0.575948,0.000000,...,0.345620,0.364095,0.191457,0.000000,1.689149,C:\Users\dorah\OneDrive\桌面\seychelles_parrot_d...,None,138.0,144.0,vdm
986,ff07fcb7-aa3b-4a2c-af70-196488a053cd,1566.5,0.082896,0.000000,1.255835,0.210454,0.515464,0.999544,0.025532,0.000000,...,0.145635,0.351407,0.214308,0.063784,2.073454,C:\Users\dorah\OneDrive\桌面\seychelles_parrot_d...,True,192.0,198.0,vdm
987,ff8b2dae-7553-4826-80bd-fb9766542ddd,1426.5,0.200742,0.000000,0.493353,0.109698,1.152392,0.015637,0.000000,0.000000,...,0.000000,0.832288,0.000000,0.590522,0.715984,C:\Users\dorah\OneDrive\桌面\seychelles_parrot_d...,True,126.0,132.0,vdm
988,ff9b1b06-4f99-43fe-b484-5a8e36e4e38c,614.5,0.204104,0.069148,1.047245,0.001327,0.751338,0.735946,0.010203,0.095548,...,0.485430,0.075930,0.138834,0.036822,1.899769,C:\Users\dorah\OneDrive\桌面\seychelles_parrot_d...,True,174.0,180.0,vdm


In [146]:
ml_features.to_csv('ml_vdm_features.csv')